# Planet Tasking API Monitoring Tasking Orders

---

## Introduction

---

This tutorial is an introduction on the bulk creation of tasking orders using [Planet](https://www.planet.com)'s Tasking API. It provides code samples on how to write simple Python code to do this.

The API reference documentation can be found at https://developers.planet.com/docs/tasking

### Requirements

---

#### Software & Modules

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Familiarity with basic REST API concepts and usage is also assumed.

We'll be using a **"Jupyter Notebook"** (aka Python Notebook) to run through the examples.
To learn more about and get started with using Jupyter, visit: [Jupyter](https://jupyter.org/) and [IPython](https://ipython.org/). 

For the best experience, download this notebook and run it on your system, and make sure to install the modules listed below first. You can also copy the examples' code to a separate Python files an run them directly with Python on your system if you prefer.

#### Planet API Key

You should have an account on the Planet Platform to access the Tasking API. You may retrieve your API key from your [account page](https://www.planet.com/account/), or from the "API Tab" in [Planet Explorer](https://www.planet.com/explorer).

## Overview

---

### The basic workflow

1. Create a bulk tasking order
1. Check the progress of the bulk tasking order

### API Endpoints

This tutorial will cover the following API ***endpoint***:

* [`/bulk`](https://api.planet.com/tasking/v2/bulk/)

## Basic Setup

---

Before interacting with the Planet Tasking API using Python, we will set up our environment with some useful modules and helper functions.

* We'll configure *authentication* to the Planet Tasking API
* We'll use the `requests` Python module to make HTTP communication easier. 
* We'll use the `json` Python module to help us work with JSON responses from the API.
* We'll use the `pytz` Python module to define the time frame for the order that we will be creating.
* We'll create a function called `p` that will print Python dictionaries nicely.

Then we'll be ready to make our first call to the Planet Tasking API by hitting the base endpoint at `https://api.planet.com/tasking/v2`. 

Let's start by configuring authentication:

### Authentication

Authentication with the Planet Tasking API can be achieved using a valid Planet **API key**.

You can *export* your API Key as an environment variable on your system:

`export PL_API_KEY="YOUR API KEY HERE"`

Or add the variable to your path, etc.

To start our Python code, we'll setup an API Key variable from an environment variable to use with our requests:

In [ ]:
# Import the os module in order to access environment variables
import os

#If you are running this notebook outside of the docker environment that comes with the repo, you can uncomment the next line to provide your API key
#os.environ['PL_API_KEY']=input('Please provide your API Key')

# Setup the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

### Helper Modules and Functions

In [ ]:
# Import helper modules
import json
import requests
import pytz
from time import sleep
from datetime import datetime, timedelta

In [ ]:
# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [ ]:
# Setup Planet Tasking PLANET_API_HOST
TASKING_API_URL = "https://api.planet.com/tasking/v2"

# Setup the session
session = requests.Session()

# Authenticate
session.headers.update({
    'Authorization': f'api-key {PLANET_API_KEY}',
    'Content-Type': 'application/json'
})

## 1 | Compose the bulk tasking order

A bulk tasking order is an asynchronous way to create many tasking orders with a single call. Once the initial POST request has been made it is then possible to make subsequent calls to be informed of the status of the request and the various orders that comprised the payload that are to be created. A bulk tasking order can be comprised of up to 1000 tasking orders of any kind that are supported by the Tasking service. 

For the purpose of this notebook we will create a handful of standard, flexible tasking orders that can then be submitted collectively viat the bulk endpoint. To make things easier we will default the start and end time of each order to start tomorrow and end 7 days from now. Of course feel free to change this to suit your needs but if you do take note that all times should be in UTC format. The start and end times are optional,but we include them in this tutorial to provide a better picture of what can be done.

In [ ]:

orders=[]

for x in range[5]
    # Define the name and coordinates for the order
    name=input("Give the order a name")
    latitude=float(input("Provide the latitude"))
    longitude=float(input("Provide the longitude"))

    # Because the geometry is GeoJSON, the coordinates must be longitude,latitude. 
    # It is also necessary to set the scheduling_type to "MONITORING"
    order = {
        'name': name,
        'geometry': {
            'type': 'Point',
            'coordinates': [
                longitude,
                latitude
            ]
        }
    }

    # Set a start and end time, giving the order a week to complete
    tomorrow = datetime.now(pytz.utc) + timedelta(days=1)
    one_week_later = tomorrow + timedelta(days=7)

    datetime_parameters = {
        'start_time': tomorrow.isoformat(),
        'end_time': one_week_later.isoformat()
    }

    # Add the monitoring parameters
    order.update(datetime_parameters)

    orders.append(order)

We then create the bulk tasking order payload using the list of orders we just defined. The bulk payload can take a pre-defined UUID as the ```id``` of the bulk tasking order, but this is optional. If it is left out then an id will be set by the system and returned as part of the ```location``` field in the response header, which we will look at later.

In [ ]:
bulk_tasking_order = {
    'order_payloads': orders
}

In [ ]:
#View the payload before posting
p(bulk_tasking_order)

In [ ]:
# The creation of an order is a POST request to the /orders endpoint
res = session.request('POST', TASKING_API_URL + '/bulk/', json=bulk_tasking_order)

if res.status_code == 403:
    print('Your PLANET_API_KEY is valid, but you are not authorized.')
elif res.status_code == 401:
    print('Your PLANET_API_KEY is incorrect')
elif res.status_code == 201:
    print('Your order was created successfully')
else:
    print(f'Received status code {res.status_code} from the API. Please contact support.')

# View the location header in the response
print(res.headers['location'])

## 2 | Check the progress of the bulk tasking order

As mentioned, a bulk tasking order is an asynchronous request so once made the builk endopint can then be polled to check the progress of the bulk tasking order as the Tasking service works its way through the orders that were provided in the payload.

The quickest way to do this is to retrieve the URL that is in the ```location``` header that is part of the response of the intial bulk POST request. This URL includes an ID which, when part of a GET request, will return the current status of the bulk request in the system:

In [ ]:
location = res.headers['location']

# The url is the entire path minus the domain
res = session.request('GET', 'https://api.planet.com' + location)

if res.status_code == 403:
    print('Your PLANET_API_KEY is valid, but you are not authorized.')
elif res.status_code == 401:
    print('Your PLANET_API_KEY is incorrect')
elif res.status_code == 201:
    print('Your order was created successfully')
else:
    print(f'Received status code {res.status_code} from the API. Please contact support.')

p(res.json())

The response to this request will show a summary of the progress that Tasking service has made in processing the bulk tasking order request. It will show the total size of the original payload: ```payload_count```, the number of tasking orders currently still being processed, the number of successfully processed tasking orders and the number of tasking orders that for one reason or another failed, these fields being ```processing_payload_count```, ```successful_payload_count``` and ```failed_payload_count``` respectively.

But what if we want to see those tasking orders that are still being processed or, more importantly, any tasking orders that failed and why they failed? This is easily done by appending "/payloads" to the end of the url that we just used in the previous GET request:

In [ ]:
# The url is the entire path minus the domain
res = session.request('GET', 'https://api.planet.com' + location + '/payloads')

if res.status_code == 403:
    print('Your PLANET_API_KEY is valid, but you are not authorized.')
elif res.status_code == 401:
    print('Your PLANET_API_KEY is incorrect')
elif res.status_code == 201:
    print('Your order was created successfully')
else:
    print(f'Received status code {res.status_code} from the API. Please contact support.')

p(res.json())

# Conclusion

As you can see, creating tasking orders in bulk is a relatively straightforward operation. It is also possible to use the ```/bulk``` endpoint to **edit** or even **cancel** multiple tasking orders at the same time. More information on how to do this can be found at https://developers.planet.com/docs/tasking/examples/bulk